<div style="display: flex;">
<img alt="fit hcmus logo" width="300" src="https://www.ctda.hcmus.edu.vn/wp-content/uploads/2022/03/FIT-logo_dat_kem_voi_logo_truong.png" onerror='this.src="https://courses.fit.hcmus.edu.vn/pluginfile.php/1/theme_academi/logo/1698750738/fit-logo-chuan-V2-MOODLE.jpg"'></img>
<div style="display: flex; flex-direction:column; justify-content: center" >
<h4 style="text-align:right; margin: 0">VNUHCM-UNIVERSITY OF SCIENCE &emsp;</h4>
<h4 style="text-align:right; margin: 0">FACULTY OF INFORMATION TECHNOLOGY &emsp;</h4>
</div>
</div>

---

# <p style="text-align:center"><b>Sinh trắc học 20-21 / HK1 / 2023-2024</b></p>
# <p style="text-align:center"><b>LAB 02 - NHẬN DẠNG BẰNG DEEP LEARNING</b></p>
<p style="text-align:center"> GVLT: Lê Hoàng Thái</p>
<p style="text-align:center"> GVTH: Lê Thanh Phong</p>

**Họ và tên:** Lê Hoài Phong</br>
**MSSV:** 20120545</br>
**SĐT:** 0387671963 </br>
**Email:** 20120545@student.hcmus.edu.vn </br>

<p style="text-align:center"> <i>Thành phố Hồ Chí Minh, ngày 10 tháng 12 năm 2023</i></p>

---

Trong file này, em thực hiện xây dựng hệ thống nhận diện realtime qua webcam, dùng thư viện face_recognition.

In [ ]:
# !pip install face_recognition

Cài đặt các logic cho hệ thống nhận diện, bao gồm việc encode các ảnh database (là kho chứa ảnh của known people); detect và nhận diện bằng camera.

In [1]:
import face_recognition
import cv2
import numpy as np
import os
import sys
import math

# Calculates the confidence score of a face match based on the face distance and a face match threshold.
def face_confidence(face_distance, threshold = 0.6):
  range = (1.0 - threshold)
  linear_val = (1.0 - face_distance) / (range * 2.0)

  if face_distance > threshold:
    return str(round(linear_val * 100, 2)) +'%'
  else:
    value = (linear_val + ((1.0 - linear_val)*math.pow((linear_val - 0.5) * 2, 0.2))) * 100
    return str(round(value, 2)) + '%'
  
class FaceRecognition:
    face_locations = [] # Vị trí mặt trên hình test
    face_encodings = [] # embedding ảnh test
    face_names = []
    known_face_encodings = [] # embedding khuôn mặt đã biết (trong database)
    known_face_names = []
    process_current_frame = True

    def __init__(self):
        pass

    # encode ảnh trong database (known face)
    def encode_faces(self, database_path):
        for name in os.listdir(database_path):
            # Tải hình và encode
            face_image = face_recognition.load_image_file(f'{database_path}/{name}')
            face_encoding = face_recognition.face_encodings(face_image)[0]
            # Lưu vào known face
            self.known_face_encodings.append(face_encoding)
            self.known_face_names.append(name.split('.')[0])

    def start(self, video_path = 0):
        video_capture = cv2.VideoCapture(video_path)

        # Nếu không mở được cam hoặc video thì thông báo lỗi
        if not video_capture.isOpened():
            sys.exit('Can not open camera/video ...')

        while True:
            _, frame = video_capture.read()

            if self.process_current_frame:
                small_frame = cv2.resize(frame, (0, 0), fx = 0.25, fy = 0.25)
                rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

                # detect mặt và encode
                self.face_locations = face_recognition.face_locations(rgb_small_frame)
                self.face_encodings = face_recognition.face_encodings(rgb_small_frame, self.face_locations)

                # So khớp từng khuôn mặt
                self.face_names = []
                for face_encoding in self.face_encodings:
                    matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                    # Mặc định là unknown face
                    name = 'Unknown'
                    confidence = 'Unknown'

                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)

                    if matches[best_match_index]:
                        name = self.known_face_names[best_match_index]
                        confidence = face_confidence(face_distances[best_match_index])

                    self.face_names.append(f'{name} ({confidence})')

            self.process_current_frame = not self.process_current_frame

            # Display annotations
            for (top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), -1)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)

            # Hiển thị frame hình đã được recognized
            cv2.imshow('Face Recognition - press q to exit', frame)

            if cv2.waitKey(1) == ord('q'):
                break

        # Tắt cam và các cửa sổ
        video_capture.release()
        cv2.waitKey(0)
        cv2.destroyAllWindows()


Khởi tạo bộ nhận diện và mở camera.

In [4]:
fr = FaceRecognition() # Khởi tạo một bộ nhận diện
fr.encode_faces('webcam_database')
fr.start()